In [42]:
import pandas as pd
from pathlib import Path
import shutil
import json
from sklearn.metrics import accuracy_score
from openai import OpenAI
from pydantic import BaseModel, Field

In [ ]:
df = pd.read_csv("data/extracted_features/extraction_evaluation/x5_llm_features_51.csv")
df.head()

In [ ]:
random_elements = df.sample(n=5, random_state=1)
random_elements

In [45]:
DATA_ROOT_PATH = Path("../../data/preprocessed/combined_data_preprocessed")

In [ ]:
for i, row in random_elements.iterrows():
    filename = f"data/extracted_features/extraction_evaluation/{row['Фамилия']}_{row['Имя']}_{row['Волна']}_LLM.json"
    features = json.loads(row["response"])
    shutil.copy(DATA_ROOT_PATH / row["Резюме"], Path.cwd())
    with open(filename, 'w') as file:
        json.dump(features, file, ensure_ascii=False, indent=4)

In [ ]:
for i, row in random_elements.iterrows():

    filename_llm = f"data/extracted_features/extraction_evaluation/{row['Фамилия']}_{row['Имя']}_{row['Волна']}_LLM.json"
    filename_true = f"data/extracted_features/extraction_evaluation/{row['Фамилия']}_{row['Имя']}_{row['Волна']}_True.json"

    with open(filename_llm, 'r') as json_file:
        features_llm = json.load(json_file)

    with open(filename_true, 'r') as json_file:
        features_true = json.load(json_file)

    accuracy = accuracy_score(list(features_true.values())[1:], list(features_llm.values())[1:])

    print(f"{row['Фамилия']}_{row['Имя']}.pdf: accuracy = {accuracy:.2f}")

**Выводы**

1. LLM очень плохо справляется с извлечением количественных и категориальных признаков с количеством категорий больше 2. Определение университета, уровня английского, высшей ступени образования и среднего балла кандидата корректно извлечь из резюме почти невозможно. 
2. С извлечением бинарных категориальных признаков дела обстоят лучше. Такие признаки, как владение Python, SQL и наличие или отсутствие в резюме информации о предыдущем опыте работы зачастую извлекаются более менее корректно. Например, если в резюме действительно нет информации об участии в конференциях, как правило, LLM корректно присваивает такому признаку значение false. Но иногда возникают проблемы с корректной интерпретацией информации, которая присутствует в резюме. Например, если в резюме указан опыт работы в качестве Computer Vision Engineer, то вполне возможно, что LLM присвоит признаку experience_as_machine_learning_engineer (есть ли у кандидата опыт работы в качестве MLE) значение false.
3. При формировании структурированного вывода возникают проблемы с валидацией признаков. В некотрых случаях LLM присваивает признакам значения, которые не удовлетворяют указанным в pydantic-схеме границам.

Таким образом, с помощью LLM более менее успешно можно извлекать только достаточно простые признаки (владение конкретными языками программирования, SQL, есть ли опыт работы в качестве DA, MLE и так далее), в то время как более значимыми являются более сложные признаки (релевантный стаж, ВУЗ, высшая ступень образования, общая структура резюме). Как следствие, по имеющимся признакам затруднительно обучить модель, которая с достаточной точностью способна была бы предсказывать вероятность прохождения технического собеседования кандидатом.